In [2]:
from datasets import load_dataset, Dataset
import torch
from torch.optim import AdamW
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    get_cosine_schedule_with_warmup,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    MistralForCausalLM,
)
from peft import (
    PeftModel,
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
)
import gc
import wandb
import yaml
import os
import time

from huggingface_hub import HfApi, CommitOperationAdd, login, hf_hub_download

In [ ]:
config = """---
hf_repo_id: hari31416/Mistral_Finance_Finetuning
start_batch_number: 0
end_batch_number: 1000
base_model_id: mistralai/Mistral-7B-Instruct-v0.1
load_pretrained: False
head_file_name: mistral_head.pt
dataset_id: gbharti/finance-alpaca
quantization_config:
  load_in_4bit: True
  bnb_4bit_quant_type: nf4
  bnb_4bit_use_double_quant: True
  bnb_4bit_compute_dtype: bfloat16
lora_config:
  r: 16
  lora_alpha: 4
  lora_dropout: 0.05
  bias: none
  task_type: CAUSAL_LM
  target_modules:
    - o_proj
    - v_proj
    - k_proj
    - q_proj
num_warmup_steps: 0
epochs: 1
max_iter_per_epoch:
max_steps:
log_interval: 1
wandb: True
project: RAGOptimize
wandb_name: fine_tune_multi_epoch
notes: RAGOptimize Training multiple epochs
lr: 0.0001
accumulation_steps: 1
batch_size: 4
max_length: 1024
model_save_root_dir: models
push_to_hub: True
push_to_hub_frequency: 100
max_hours: 11.7
"""
config = yaml.safe_load(config)
# Create the model save root directory so that we can save the model
os.makedirs(config["model_save_root_dir"], exist_ok=True)


OSError: [Errno 30] Read-only file system: '/kaggle'